In [ ]:
%pip install pandas
import pandas as pd
import zipfile
zipfile_path = 'Datasets_final.zip'
with zipfile.ZipFile(zipfile_path, 'r') as file_ref:
    file_ref.extractall("data") 
    train_data = pd.read_csv(file_ref.open('train_dataset_final1.csv'))
    validate_data = pd.read_csv(file_ref.open('validate_dataset_final.csv'))
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_data.head(10).T

In [ ]:
train_data['next_month_default'].value_counts()

In [ ]:
train_data.sample().T

In [ ]:
validate_data.sample(5).T

In [ ]:
train_data.isnull().sum()/train_data.shape[0]

In [ ]:
train_data.dropna(inplace = True)


In [ ]:
train_data.describe().T

In [ ]:
corr = train_data.corr()
%pip install seaborn
import seaborn as sns
sns.heatmap(corr, cmap = 'YlGnBu')

In [ ]:
train_data.groupby('marriage')['next_month_default'].mean().plot(kind = 'bar')

In [ ]:
train_data.marriage.value_counts()

In [ ]:
train_data = train_data.drop(train_data[train_data['marriage'] == 0].index)

In [ ]:
train_data.groupby('marriage')['next_month_default'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('sex')['next_month_default'].mean().plot(kind = 'bar')

In [ ]:
train_data.sex.value_counts()

In [ ]:
train_data.groupby('education')['next_month_default'].mean().plot(kind = 'bar')

In [ ]:
train_data = train_data.drop(train_data[train_data['education'] == 0].index )

In [ ]:
train_data = train_data.drop(train_data[train_data['education'] > 4].index )

In [ ]:
train_data.groupby('education')['next_month_default'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('pay_0')['next_month_default'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('pay_2')['next_month_default'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('pay_3')['next_month_default'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('pay_4')['next_month_default'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('pay_5')['next_month_default'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('pay_6')['next_month_default'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['Bill_amt1'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['Bill_amt2'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['Bill_amt3'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['Bill_amt4'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['Bill_amt5'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['Bill_amt6'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['pay_amt1'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['pay_amt2'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['pay_amt3'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['pay_amt4'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['pay_amt5'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['pay_amt6'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('sex')['education'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('sex')['PAY_TO_BILL_ratio'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('education')['PAY_TO_BILL_ratio'].mean().plot(kind = 'bar')

In [ ]:
train_data.groupby('marriage')['PAY_TO_BILL_ratio'].mean().plot(kind = 'bar')

In [ ]:
train_data[train_data['education'] > 3].shape

In [ ]:
train_data[train_data['education'] <= 3].shape

In [ ]:
output = train_data['next_month_default']
train_data.drop(['next_month_default'], axis = 1, inplace = True)
train_data.shape

In [ ]:
train_data['credit_util_ratio'] = train_data['AVG_Bill_amt'] / (train_data['LIMIT_BAL'] + 1e-6)
pay_cols = ['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']
train_data['delinquency_count'] = (train_data[pay_cols] >= 1).sum(axis=1)
bill_cols = [f'Bill_amt{i}' for i in range(1, 7)]
train_data['overpayment_count'] = (train_data[bill_cols] < 0).sum(axis=1)
def longest_streak(row):
    streak = max_streak = 0
    for val in row:
        if val >= 1:
            streak += 1
            max_streak = max(max_streak, streak)
        else:
            streak = 0
    return max_streak

train_data['longest_delinquency_streak'] = train_data[pay_cols].apply(longest_streak, axis=1)
train_data['percent_fully_paid_months'] = (train_data[pay_cols] == -1).sum(axis=1) / len(pay_cols)

train_data.shape

In [ ]:
x = train_data
y = output
%pip install imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
%pip install scikit-learn
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
y_train = y_train.astype(int)
x_resampled, y_resampled = smote.fit_resample(x_train, y_train)

In [ ]:
x_resampled.shape

In [ ]:
y_resampled.value_counts()

In [ ]:
%pip install lightgbm
from lightgbm import LGBMClassifier


In [ ]:
lgbm = LGBMClassifier(n_estimators=100, max_depth=3, random_state=42)
lgbm.fit(x_resampled, y_resampled)
feature_imp = pd.Series(lgbm.feature_importances_, index=x_resampled.columns)
top_features = feature_imp.sort_values(ascending=False).head(10).index.tolist()
print(top_features)
x_train_sub = x_resampled[top_features]
x_test_sub = x_test[top_features]


In [ ]:
from sklearn.linear_model import LogisticRegression
param_grid = {
    'penalty': ['elasticnet'],
    'C': [0.001],
    'solver': ['saga'],
    'max_iter': [500],
    'l1_ratio':[0.5]
}
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(LogisticRegression(max_iter = 100), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(x_train_sub, y_resampled)

best_tree = grid.best_estimator_
y_pred_lr = best_tree.predict(x_test_sub)
y_probs = best_tree.predict_proba(x_test_sub)[:, 1]
%pip install numpy
import numpy as np
from sklearn.metrics import fbeta_score
thresholds = np.arange(0.1, 0.9, 0.01)
f2_scores = []
best_threshold = 0.5
best_f2 = 0
for t in thresholds:
    y_pred = (y_probs > t).astype(int)
    score = fbeta_score(y_test, y_pred, beta=2.5)
    f2_scores.append(score)
    if score > best_f2:
        best_f2 = score
        best_threshold = t
y_pred_final = (y_probs > best_threshold).astype(int)
print(f"Best F2 score: {best_f2:.2f} at threshold: {best_threshold:.2f}")
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_final))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [10],
    'min_samples_split': [5],
    'min_samples_leaf': [2],
    'max_features': ['sqrt'],
    'criterion': ['gini']
}


grid = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(x_train_sub, y_resampled)

best_tree = grid.best_estimator_
y_pred_dt = best_tree.predict(x_test_sub)
y_probs = best_tree.predict_proba(x_test_sub)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f2_scores = []
best_threshold = 0.5
best_f2 = 0
for t in thresholds:
    y_pred = (y_probs > t).astype(int)
    score = fbeta_score(y_test, y_pred, beta=2.5)
    f2_scores.append(score)
    if score > best_f2:
        best_f2 = score
        best_threshold = t
y_pred_final = (y_probs > best_threshold).astype(int)
print(f"Best F2 score: {best_f2:.2f} at threshold: {best_threshold:.2f}")
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_final))

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100],
    'learning_rate': [0.1],
    'max_depth': [7],
    'num_leaves': [31],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'reg_alpha': [0.1],
    'reg_lambda': [0.1],
    'scale_pos_weight': [5]
}



grid = GridSearchCV(LGBMClassifier(), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(x_train_sub, y_resampled)

best_tree_lgbm = grid.best_estimator_
y_pred_lgbm = best_tree_lgbm.predict(x_test_sub)
from sklearn.metrics import fbeta_score
y_probs = best_tree_lgbm.predict_proba(x_test_sub)[:, 1]
import numpy as np
thresholds = np.arange(0.1, 0.9, 0.01)
f2_scores = []
best_threshold = 0.5
best_f2 = 0
for t in thresholds:
    y_pred = (y_probs > t).astype(int)
    score = fbeta_score(y_test, y_pred, beta=2.5)
    f2_scores.append(score)
    if score > best_f2:
        best_f2 = score
        best_threshold = t
y_pred_final = (y_probs > best_threshold).astype(int)
print(f"Best F2 score: {best_f2:.2f} at threshold: {best_threshold:.2f}")
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_final))

In [ ]:
%pip install xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(x_train_sub, y_resampled)

In [ ]:
param_grid = {
    'n_estimators': [200],
    'max_depth': [7],
    'learning_rate': [0.01],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'gamma': [1],
    'min_child_weight': [3],
    'scale_pos_weight': [3]
}
grid_xgb = GridSearchCV(XGBClassifier(), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_xgb.fit(x_train_sub, y_resampled)

best_tree_xgb = grid_xgb.best_estimator_
y_pred_xgb = best_tree_xgb.predict(x_test_sub)
y_probs = best_tree_xgb.predict_proba(x_test_sub)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f2_scores = []
best_threshold = 0.5
best_f2 = 0
for t in thresholds:
    y_pred = (y_probs > t).astype(int)
    score = fbeta_score(y_test, y_pred, beta=2.5)
    f2_scores.append(score)
    if score > best_f2:
        best_f2 = score
        best_threshold = t
y_pred_final = (y_probs > best_threshold).astype(int)
print(f"Best F2 score: {best_f2:.2f} at threshold: {best_threshold:.2f}")
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_final))

In [ ]:
validate_data['credit_util_ratio'] = validate_data['AVG_Bill_amt'] / (validate_data['LIMIT_BAL'] + 1e-6)
pay_cols = ['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']
validate_data['delinquency_count'] = (validate_data[pay_cols] >= 1).sum(axis=1)
bill_cols = [f'Bill_amt{i}' for i in range(1, 7)]
validate_data['overpayment_count'] = (validate_data[bill_cols] < 0).sum(axis=1)
def longest_streak(row):
    streak = max_streak = 0
    for val in row:
        if val >= 1:
            streak += 1
            max_streak = max(max_streak, streak)
        else:
            streak = 0
    return max_streak

validate_data['longest_delinquency_streak'] = validate_data[pay_cols].apply(longest_streak, axis=1)
validate_data['percent_fully_paid_months'] = (validate_data[pay_cols] == -1).sum(axis=1) / len(pay_cols)
validate_data_sub = validate_data[top_features]
val_probs = best_tree_lgbm.predict_proba(validate_data_sub)[:, 1]
val_preds = (val_probs > best_threshold).astype(int)
submission = pd.DataFrame({
    'Customer': validate_data['Customer_ID'],
    'next_month_default': val_preds
})
submission.to_csv('submission_23124020.csv', index=False)


In [ ]:
import joblib

# Ensure you have your best model from the grid search
# best_tree_lgbm is the variable name from your notebook
joblib.dump(best_tree_lgbm, 'credit_risk_model.pkl')

print("Model saved successfully!")